In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%%javascript
$('#menubar').hide();

In [2]:
import os

In [3]:
import plotly.offline as offline
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly import tools
offline.init_notebook_mode(connected=True)

SLICING = 10

def create_graphs(data_frames, **kwargs):
    data = [        
            go.Scatter(
                x=data_frames[df_index].iloc[0::SLICING,0],
                y=data_frames[df_index].iloc[0::SLICING,i],
                #opacity=1, 
                xaxis=("x" + str(df_index + 1)),
                yaxis=("y" + str(df_index + 1)),
                name=str(data_frames[df_index].columns[i])
            ) for df_index in range(0, len(data_frames)) 
            for i in range(1,data_frames[df_index].columns.size)
    ]
    
    layout = go.Layout(**kwargs)
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False, "showLink":False})

def create_graph(data_frame, title=None, x_axis_title=None, y_axis_title = None):
    data = [        
            go.Scatter(
                x=data_frame.iloc[0::SLICING,0],
                y=data_frame.iloc[0::SLICING,i],
                #opacity=1,
                name=str(data_frame.columns[i])
            ) 
            for i in range(1,data_frame.columns.size)
            
    ]
    
    #fig = tools.make_subplots(rows=1, cols=len(data_frames))
    layout = go.Layout(
        title=title,         
        showlegend=False,
        xaxis=dict(
            title=x_axis_title
        ),
        yaxis=dict(
            title=y_axis_title
        )        
    )
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False, "showLink":False})

In [4]:
from simcore_sdk.nodeports.nodeports import PORTS

In [5]:
# from plotall.m

import pandas as pd

data_path_ty = PORTS.inputs[0].get()
data_frame_ty = pd.read_csv(data_path_ty, sep='\t', header=None)

# scale time
f = lambda x: x/1000.0
data_frame_ty[0] = data_frame_ty[0].apply(f)
syids = 9
yids = [30, 31, 32, 33, 34, 36, 37, 38, 39]
ynid = [0] * 206
for id in range(1,syids):
    ynid[yids[id]] = id

data_path_ar = PORTS.inputs[1].get()
data_frame_ar = pd.read_csv(data_path_ar, sep='\t', header=None)

tArray = 1
I_Ca_store = 2
Ito = 3
Itof = 4
Itos = 5
INa = 6
IK1 = 7
s1 = 8
k1 = 9
Jserca = 10
Iks = 11
Ikr = 12
Jleak = [13,14]
ICFTR = 15
Incx = 16

In [6]:
# membrane potential
title="Membrane Potential"
axis_colums = [0,ynid[39]+1]
plot_data = data_frame_ty.filter(items=[data_frame_ty.columns[i] for i in axis_colums])
create_graph(data_frame=plot_data, 
             x_axis_title="time (sec)",
             y_axis_title=title)

In [7]:
# LCC current (ICa)
title="I<sub>Ca</sub> (pA/pF)"
axis_colums = [0,I_Ca_store-1]
plot_data = data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums])
create_graph(data_frame=plot_data, 
             x_axis_title="time (sec)",
             y_axis_title=title)


In [8]:
# CaSRT & Caj
data_frame_casrt = data_frame_ty.filter(items=[data_frame_ty.columns[0], data_frame_ty.columns[ynid[30]+1], data_frame_ty.columns[ynid[31]+1]])
data_frame_casrt[3] = data_frame_casrt[1] + data_frame_casrt[2]
plot_data = [data_frame_casrt.filter(items=[data_frame_casrt.columns[0], data_frame_casrt.columns[3]])]

# 
g = lambda x: x*1000.0
axis_colums = [0,ynid[36]+1]
data_frame_ty[ynid[36]+1] = data_frame_ty[ynid[36]+1].apply(g)
plot_data.append(data_frame_ty.filter(items=[data_frame_ty.columns[i] for i in axis_colums]))

axis_colums = [0,ynid[37]+1]
plot_data.append(data_frame_ty.filter(items=[data_frame_ty.columns[i] for i in axis_colums]))
create_graphs(data_frames=plot_data, 
              title=None, 
              showlegend=False,
              xaxis=dict(
                  domain=[0,0.3],
                  title="time (sec)"
              ),
              xaxis2=dict(
                  domain=[0.4,0.6],
                  title="time (sec)"),
              xaxis3=dict(
                  domain=[0.7,1.0],
                  title="time (sec)"),
              yaxis=dict(
                  title="[Ca]<sub>SRT</sub> (mM)"
              ),
              yaxis2=dict(
                  title="Ca Dyad (\u00B5M)", 
                  anchor="x2"),
              yaxis3=dict(
                  title="Ca sl (mM)", 
                  anchor="x3")
             )

In [9]:
# Cai
title="[Ca]<sub>i</sub> (\u00B5M)"
axis_colums = [0,ynid[38]+1]
plot_data = data_frame_ty.filter(items=[data_frame_ty.columns[i] for i in axis_colums])
create_graph(data_frame=plot_data, 
             title=None, 
             x_axis_title="time (sec)",
             y_axis_title=title)

In [10]:
# Ito
title="I<sub>to</sub> (pA/pF)"
axis_colums = [0,Ito-1]
plot_data = data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums])
create_graph(data_frame=plot_data, 
             title=None, 
             x_axis_title="time (sec)",
             y_axis_title=title)


In [11]:
# INa
title="I<sub>Na</sub> (pA/pF)"
axis_colums = [0,INa-1]
plot_data = data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums])
create_graph(data_frame=plot_data, 
             title=None, 
             x_axis_title="time (sec)",
             y_axis_title=title)


In [12]:
# IKs and ICFTR
axis_colums = [0,Iks-1]
plot_data = [data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums])]

axis_colums = [0,ICFTR-1]
plot_data.append(data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums]))

create_graphs(data_frames=plot_data, 
              title=None, 
              showlegend=False,
              #xaxis=dict(title="time (sec)"),
              xaxis2=dict(title="time (sec)", anchor="y2"),
              yaxis=dict(
                  domain=[0.6,1.0],
                  title="I<sub>Ks</sub> (pA/pF)"
              ),
              yaxis2=dict(
                  domain=[0,0.5],
                  title="I<sub>CFTR</sub>", 
                  anchor="x2")
             )

In [13]:
# IKr and IK1
axis_colums = [0,Ikr-1]
plot_data = [data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums])]

axis_colums = [0,IK1-1]
plot_data.append(data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums]))

create_graphs(data_frames=plot_data, 
              title=None, 
              showlegend=False,
              #xaxis=dict(title="time (sec)"),
              xaxis2=dict(title="time (sec)", anchor="y2"),
              yaxis=dict(
                  domain=[0.6,1.0],
                  title="I<sub>Kr</sub> (pA/pF)"
              ),
              yaxis2=dict(
                  domain=[0,0.5],
                  title="I<sub>K1</sub> (pA/pF)", 
                  anchor="x2")
             )

In [14]:
# [Na]
axis_colums = [0,ynid[32]+1]
plot_data = [data_frame_ty.filter(items=[data_frame_ty.columns[i] for i in axis_colums])]

axis_colums = [0,ynid[33]+1]
plot_data.append(data_frame_ty.filter(items=[data_frame_ty.columns[i] for i in axis_colums]))

axis_colums = [0,ynid[34]+1]
plot_data.append(data_frame_ty.filter(items=[data_frame_ty.columns[i] for i in axis_colums]))

create_graphs(data_frames=plot_data, 
              title=None, 
              showlegend=False,
              xaxis=dict(title="time (sec)", domain=[0,0.3]),
              xaxis2=dict(title="time (sec)", domain=[0.4,0.6]),
              xaxis3=dict(title="time (sec)", domain=[0.7,1.0]),
              yaxis=dict(
                  title="[Na]<sub>j</sub>"
              ),
              yaxis2=dict(
                  title="[Na]<sub>s<sup>l</sup></sub>", 
                  anchor="x2"),
              yaxis3=dict(
                  title="[Na]<sub>i</sub> (mmol/L relevant compartment", 
                  anchor="x3")
             )

In [15]:
# I_NCX
title="I<sub>NCX</sub> (pA/pF)"
axis_colums = [0,Incx-1]
plot_data = data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums])
create_graph(data_frame=plot_data, 
             title=None, 
             x_axis_title="time (sec)",
             y_axis_title=title)


In [16]:
# RyR fluxes
axis_colums = [0,Jleak[0]-1]
plot_data = [data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums])]

axis_colums = [0,Jleak[1]-1]
plot_data.append(data_frame_ar.filter(items=[data_frame_ar.columns[i] for i in axis_colums]))

sr_ca_release = data_frame_ar.filter(items=[data_frame_ar.columns[0]])
sr_ca_release[1] = data_frame_ar[Jleak[0]-1] - data_frame_ar[Jleak[1]-1]
plot_data.append(sr_ca_release)
create_graphs(data_frames=plot_data, 
              title=None, 
              showlegend=False,
              xaxis=dict(title=None),
              xaxis2=dict(title=None),
              xaxis3=dict(title="time (sec)", anchor="y3"),
              yaxis=dict(
                  domain=[0.7,1.0],
                  title="JRyR<sub>tot</sub>"
              ),
              yaxis2=dict(
                  domain=[0.4,0.6],
                  title="Passive Leak", 
                  anchor="x2"),
              yaxis3=dict(
                  domain=[0,0.3],
                  title="SR Ca release", 
                  anchor="x3")
             )